In [ ]:
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv("final_discounted_heavy.csv")
df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'], errors='coerce')
df = df.dropna(subset=['ORDERDATE'])

# Add Year column
df['Year'] = df['ORDERDATE'].dt.year

# Simulate Customer Tiers
np.random.seed(42)
df['CustomerTier'] = np.random.choice(['Gold', 'New', 'Regular'], size=len(df), p=[0.2, 0.3, 0.5])

# Simulate Product Type
df['ProductType'] = np.where(df['PRODUCTLINE'].isin(['Classic Cars', 'Vintage Cars']), 'Premium', 'Budget')

# Smart Discount Strategy
def apply_discount(row):
    if row['CustomerTier'] == 'Gold':
        return 0.10
    elif row['CustomerTier'] == 'New':
        return 0.05
    elif row['ProductType'] == 'Premium':
        return 0.05
    else:
        return 0.10

df['SmartDiscount'] = df.apply(apply_discount, axis=1)

# Adjust sales based on discount
df['SmartSale'] = df['SALES'] * (1 - df['SmartDiscount'])

# Smart Shipping Strategy
df['SmartShippingFactor'] = np.where(df['CustomerTier'] == 'Gold', 1.0, 0.95)
df['SmartCost'] = df['EstimatedCost'] * df['SmartShippingFactor']

# Recalculate profit
df['SmartProfit'] = df['SmartSale'] - df['SmartCost']

# Yearly Aggregation
summary = df.groupby('Year').agg({
    'SmartSale': 'sum',
    'SmartCost': 'sum',
    'SmartProfit': 'sum'
}).reset_index()

# Margins
summary['GrossProfitMargin'] = (summary['SmartProfit'] / summary['SmartSale']).round(2)

summary.to_csv("Smart_Strategy_Financials.csv", index=False)

print(summary.round(2))
